In [18]:
import cv2
import numpy as np
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu

# Load the MRI image (replace 'your_image.jpg' with the actual file path)
image = cv2.imread(r'W:\University of Moratuwa\Academics\GitHub\Image-Processing-and-Machine-Vision\Project\Dataset_Images\1.png', 0)

# Pre-process the image
image = cv2.GaussianBlur(image, (5, 5), 0)  # Apply Gaussian blur to reduce noise
#threshold_value = threshold_otsu(image)
#, binary_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
_, binary_image = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)
# Label connected components
labeled_image = label(binary_image)
cleared_image = clear_border(labeled_image)

# Find regions (tumor candidates) in the image
regions = regionprops(cleared_image)

# Define a function to calculate region properties (e.g., area)
def calculate_properties(region):
    min_row, min_col, max_row, max_col = region.bbox
    return {
        "area": region.area,
        "min_row": min_row,
        "min_col": min_col,
        "max_row": max_row,
        "max_col": max_col,
    }

# Define a minimum tumor area threshold (adjust as needed)
min_tumor_area = 100  # You may need to experiment with different values

# Filter and segment tumors based on region properties (e.g., area)
tumor_regions = [r for r in regions if r.area > min_tumor_area]

# Create a mask for the tumor regions
mask = np.zeros_like(image)
for region in tumor_regions:
    min_row, min_col, max_row, max_col = region.bbox
    mask[min_row:max_row, min_col:max_col] = 1

# Apply the mask to the original image
segmented_image = image * mask

# Display the original image and segmented result
cv2.imshow("Original Image", image)
cv2.imshow("Segmented Image", segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the MRI image
image = cv2.imread("brain_mri.jpg", 0)  # Load the image in grayscale

# Pre-processing (if needed)
# For example, you can apply Gaussian blur to reduce noise:
# image = cv2.GaussianBlur(image, (5, 5), 0)

# Thresholding to detect potential tumor regions
_, binary_image = cv2.threshold(image, 170, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize an empty mask for the segmented tumor region
segmented_mask = np.zeros_like(image)

# Draw the contours on the mask to segment the tumor
cv2.drawContours(segmented_mask, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

# Apply the mask to the original image for visualization
segmented_image = cv2.bitwise_and(image, segmented_mask)

# Display the original image, segmented mask, and segmented image
cv2.imshow("Original Image", image)
cv2.imshow("Segmented Mask", segmented_mask)
cv2.imshow("Segmented Image", segmented_image)

# Save the segmented image if needed
# cv2.imwrite("segmented_brain.jpg", segmented_image)

cv2.waitKey(0)
cv2.destroyAllWindows()
